In [1]:
from tqdm import tqdm
from pprint import pprint
import json
import pandas as pd
import sys
sys.path.append('/home/ajassani/iLoveTrace')
from Trace2Tree.trace_to_tree import TraceToTree
from tree_perf import TreePerfAnalyzer

In [2]:
path = '/home/ajassani/jan22_2025/AMD_Instinct_MI325X_llama3_8b_fp8_bsz_9_trace/quanta-cyxtera-r35a-3_143814.1736558515673261472.pt.trace.json'
with open(path, 'r') as f:
    data = json.load(f)

events = data['traceEvents']
tree = TraceToTree(events)
tree.build_tree(add_python_func=False)
perf_analyzer = TreePerfAnalyzer(tree)

Building tree with add_python_func=False
Building CPU op tree with add_python_func=False


In [3]:
gemm_events = [event for event in tree.events if event['name'] in ['aten::addmm', 'aten::mm', 'tex_ts::te_gemm_ts']]
gemm_events_uids = [event['UID'] for event in gemm_events]
df_gemm_ops = perf_analyzer.build_df_perf_metrics(gemm_events_uids, bwd=False, non_data_mov=True)
perf_analyzer.summarize_df_perf_metrics(df_gemm_ops, ['mean'])

,name,param: M,param: N,param: K,param: bias,GFLOPS_first,FLOPS/Byte_first,TFLOPS/s_mean,Non-Data-Mov TFLOPS/s_mean,Non-Data-Mov Kernel Time (µs)_sum,Kernel Time (µs)_sum,name_count
7,tex_ts::te_gemm_ts,36864,4096,28672,False,8658.654069,3266.430380,1257.332407,1257.332407,440972.674,440972.674,64
3,tex_ts::te_gemm_ts,28672,4096,36864,False,8658.654069,3266.430380,1293.015710,1293.015710,428768.744,428768.744,64
11,tex_ts::te_gemm_ts,36864,28672,4096,False,8658.654069,3266.430380,1301.100948,1301.100948,425938.543,425938.543,64
10,tex_ts::te_gemm_ts,36864,14336,4096,False,4329.327034,2932.363636,1101.318475,1101.318475,251764.024,251764.024,64
1,tex_ts::te_gemm_ts,4096,14336,36864,False,4329.327034,2932.363636,1154.742689,1154.742689,240355.766,240355.766,64
6,tex_ts::te_gemm_ts,36864,4096,14336,False,4329.327034,2932.363636,1346.705884,1346.705884,205778.997,205778.997,64
9,tex_ts::te_gemm_ts,36864,5120,4096,False,1546.188227,2143.255814,634.641402,634.641402,158751.164,158751.164,64
4,tex_ts::te_gemm_ts,36864,4096,4096,False,1236.950581,1940.210526,1235.232125,1235.232125,128446.104,128446.104,128
2,tex_ts::te_gemm_ts,5120,4096,36864,False,1546.188227,2143.255814,1358.900470,1358.900470,72832.521,72832.521,64
5,tex_ts::te_gemm_ts,36864,4096,5120,False,1546.188227,2143.255814,1386.001948,1386.001948,71419.724,71419.724,64
